In [13]:
import pandas as pd
from nltk.tag import pos_tag
import re
from collections import defaultdict,Counter
from nltk.stem import WordNetLemmatizer
from datetime import datetime
from tqdm import tqdm
import numpy as np
import os
tqdm.pandas()
from ast import literal_eval
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
from urllib import request
import pandas as pd
module_url = f"https://media.githubusercontent.com/media/Ai-Adventures/Test_Datasets/main/ML_Test_1.csv"
module_name = 'ML_Test_1.csv'
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
df = pd.read_csv('ML_Test_1.csv')
df = df.fillna('')

Fetching https://media.githubusercontent.com/media/Ai-Adventures/Test_Datasets/main/ML_Test_1.csv


In [15]:
df.head()

,author,posted_at,num_comments,score,selftext,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts,subreddit,title,total_awards_received,upvote_ratio,user_num_posts,user_registered_at,user_upvote_ratio
0,-Howitzer-,2020-08-17 20:26:04,19,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,BREAKING: Trump to begin hiding in mailboxes t...,0,1.00,4661,2012-11-09,-0.658599
1,-Howitzer-,2020-07-06 17:01:48,1,3,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Joe Biden's America,0,0.67,4661,2012-11-09,-0.658599
2,-Howitzer-,2020-09-09 02:29:02,3,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,4 more years and we can erase his legacy for g...,0,1.00,4661,2012-11-09,-0.658599
3,-Howitzer-,2020-06-23 23:02:39,2,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Revelation 9:6 [Transhumanism: The New Religio...,0,1.00,4661,2012-11-09,-0.658599
4,-Howitzer-,2020-08-07 04:13:53,32,622,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,"LOOK HERE, FAT",0,0.88,4661,2012-11-09,-0.658599


The column subr_faved_by contains an array of values (names of redditors who added the subreddit to which the current post was submitted),  they are in text format,convert these string values to Python lists, going from

In [16]:
import ast
def transform_faves(df):
   df['subr_faved_by_as_list'] = df['subr_faved_by'].map(lambda x: ast.literal_eval(x))
   return df
df = transform_faves(df)
df

,author,posted_at,num_comments,score,selftext,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts,subreddit,title,total_awards_received,upvote_ratio,user_num_posts,user_registered_at,user_upvote_ratio,subr_faved_by_as_list
0,-Howitzer-,2020-08-17 20:26:04,19,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,BREAKING: Trump to begin hiding in mailboxes t...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo..."
1,-Howitzer-,2020-07-06 17:01:48,1,3,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Joe Biden's America,0,0.67,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo..."
2,-Howitzer-,2020-09-09 02:29:02,3,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,4 more years and we can erase his legacy for g...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo..."
3,-Howitzer-,2020-06-23 23:02:39,2,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Revelation 9:6 [Transhumanism: The New Religio...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo..."
4,-Howitzer-,2020-08-07 04:13:53,32,622,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,"LOOK HERE, FAT",0,0.88,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19935,zqrwiel,2020-07-23 16:39:15,11,246,,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,carti why,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe..."
19936,zqrwiel,2020-12-15 11:25:07,39,1,"Then I think we might get 18 songs, outro usua...",2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,If uzi on track 3 and 16,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe..."
19937,zqrwiel,2020-12-27 13:57:49,15,1,He has 25songs to perform plus the additional ...,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,Man carti’s concerts are gonna be long af,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe..."
19938,zqrwiel,2020-12-29 12:07:10,6,1,I got goose[***]ps just by thinking about it 😬,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,Can’t wait to see Carti going full rage mode o...,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe..."


All Reddit posts need to have a title, but a text body is optional. However, we want to be able to access all free text information for each post without having to look at two columns every time

In [17]:
def concat(df):
    def is_missing(x):
        if pd.isna(x):
            return True
        s = str(x).strip()
        return s == "" or len(s) == 1
    df['title'] = df['title'].apply(lambda x: f"<title>{x}</title>" if not is_missing(x) else '' )
    df['selftext'] = df['selftext'].apply(lambda x: f"<selftext>{x}</selftext>" if not is_missing(x) else '')
    df['full_text'] = df.apply(lambda row: f"{row['title']}\n{row['selftext']}" if row['title'] and row['selftext']else (row['title'] or row['selftext']),axis=1)
    return df
df = concat(df)
df

,author,posted_at,num_comments,score,selftext,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts,subreddit,title,total_awards_received,upvote_ratio,user_num_posts,user_registered_at,user_upvote_ratio,subr_faved_by_as_list,full_text
0,-Howitzer-,2020-08-17 20:26:04,19,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,<title>BREAKING: Trump to begin hiding in mail...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>BREAKING: Trump to begin hiding in mail...
1,-Howitzer-,2020-07-06 17:01:48,1,3,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,<title>Joe Biden's America</title>,0,0.67,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>Joe Biden's America</title>
2,-Howitzer-,2020-09-09 02:29:02,3,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,<title>4 more years and we can erase his legac...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>4 more years and we can erase his legac...
3,-Howitzer-,2020-06-23 23:02:39,2,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,<title>Revelation 9:6 [Transhumanism: The New ...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>Revelation 9:6 [Transhumanism: The New ...
4,-Howitzer-,2020-08-07 04:13:53,32,622,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,"<title>LOOK HERE, FAT</title>",0,0.88,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...","<title>LOOK HERE, FAT</title>"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19935,zqrwiel,2020-07-23 16:39:15,11,246,,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,<title>carti why</title>,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe...",<title>carti why</title>
19936,zqrwiel,2020-12-15 11:25:07,39,1,"<selftext>Then I think we might get 18 songs, ...",2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,<title>If uzi on track 3 and 16</title>,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe...",<title>If uzi on track 3 and 16</title>\n<self...
19937,zqrwiel,2020-12-27 13:57:49,15,1,<selftext>He has 25songs to perform plus the a...,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,<title>Man carti’s concerts are gonna be long ...,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe...",<title>Man carti’s concerts are gonna be long ...
19938,zqrwiel,2020-12-29 12:07:10,6,1,<selftext>I got goose[***]ps just by thinking ...,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,<title>Can’t wait to see Carti going full rage...,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe...",<title>Can’t wait to see Carti going full rage...


We would like to augment our text data with linguistic information. To this end, we will tokenize, apply part-of-speech tagging, and then we will lower case all the posts.

In [18]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [19]:
import re
def enrich_posts(df):
 def process_text(text):
  tokens = re.split(r'\W+', text)
  tokens = [t for t in tokens if t]
  tagged = pos_tag(tokens)
  tagged_lower = []
  for w, t in tagged:
    tagged_lower.append((w.lower(), t))
  return tagged_lower
 df['enriched_title'] = df['title'].apply(process_text)
 df['enriched_selftext'] = df['selftext'].apply(process_text)
 return df
df = enrich_posts(df)
df

,author,posted_at,num_comments,score,selftext,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts,...,title,total_awards_received,upvote_ratio,user_num_posts,user_registered_at,user_upvote_ratio,subr_faved_by_as_list,full_text,enriched_title,enriched_selftext
0,-Howitzer-,2020-08-17 20:26:04,19,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,<title>BREAKING: Trump to begin hiding in mail...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>BREAKING: Trump to begin hiding in mail...,"[(title, NN), (breaking, NNP), (trump, NNP), (...",[]
1,-Howitzer-,2020-07-06 17:01:48,1,3,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,<title>Joe Biden's America</title>,0,0.67,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>Joe Biden's America</title>,"[(title, NN), (joe, NNP), (biden, NNP), (s, NN...",[]
2,-Howitzer-,2020-09-09 02:29:02,3,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,<title>4 more years and we can erase his legac...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>4 more years and we can erase his legac...,"[(title, NN), (4, CD), (more, JJR), (years, NN...",[]
3,-Howitzer-,2020-06-23 23:02:39,2,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,<title>Revelation 9:6 [Transhumanism: The New ...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>Revelation 9:6 [Transhumanism: The New ...,"[(title, NN), (revelation, NNP), (9, CD), (6, ...",[]
4,-Howitzer-,2020-08-07 04:13:53,32,622,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,"<title>LOOK HERE, FAT</title>",0,0.88,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...","<title>LOOK HERE, FAT</title>","[(title, NN), (look, NNP), (here, NNP), (fat, ...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19935,zqrwiel,2020-07-23 16:39:15,11,246,,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,...,<title>carti why</title>,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe...",<title>carti why</title>,"[(title, NN), (carti, NN), (why, WRB), (title,...",[]
19936,zqrwiel,2020-12-15 11:25:07,39,1,"<selftext>Then I think we might get 18 songs, ...",2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,...,<title>If uzi on track 3 and 16</title>,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe...",<title>If uzi on track 3 and 16</title>\n<self...,"[(title, NN), (if, IN), (uzi, JJ), (on, IN), (...","[(selftext, JJ), (then, RB), (i, PRP), (think,..."
19937,zqrwiel,2020-12-27 13:57:49,15,1,<selftext>He has 25songs to perform plus the a...,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,...,<title>Man carti’s concerts are gonna be long ...,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe...",<title>Man carti’s concerts are gonna be long ...,"[(title, NN), (man, NNP), (carti, VBZ), (s, JJ...","[(selftext, NN), (he, PRP), (has, VBZ), (25son..."
19938,zqrwiel,2020-12-29 12:07:10,6,1,<selftext>I got goose[***]ps just by thinking ...,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,...,<title>Can’t wait to see Carti going full rage...,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe...",<title>Can’t wait to see Carti goin

 Users with best scores

In [20]:
def user_best_score(df):
   grp= df.groupby('author')['score'].sum()
   cu = grp[grp > 10000].sort_values(ascending=False)
   my_dict = cu.to_dict()
   return my_dict

mg = user_best_score(df)
mg

{'DaFunkJunkie': 250375,
 '': 218846,
 'SUPERGUESSOUS': 211611,
 'jigsawmap': 210824,
 'chrisdh79': 143538,
 'hildebrand_rarity': 122464,
 'iSlingShlong': 118595,
 'hilltopye': 81245,
 'tefunka': 79560,
 'OldFashionedJizz': 64398,
 'JLBesq1981': 58235,
 'rspix000': 57107,
 'Wagamaga': 47989,
 'stem12345679': 47455,
 'TheJeck': 26058,
 'TheGamerDanYT': 25357,
 'TrumpSharted': 21154,
 'NotsoPG': 18518,
 'SonictheManhog': 18116,
 'BlanketMage': 13677,
 'NewAltWhoThis': 12771,
 'kevinmrr': 11900,
 'Dajakesta0624': 11613,
 'apocalypticalley': 10382}

 Awarded posts

In [21]:
df[df['total_awards_received'] >= 1].max()['user_num_posts']


np.int64(9977)

In [22]:
df.loc[df['total_awards_received'] >= 1]['user_num_posts']

,user_num_posts
67,1105
69,1105
818,8292
866,6102
911,6900
...,...
17041,2476
17361,4838
18599,3862
18823,4604


In [23]:
def Awarded_posts(df):
    post=(df['total_awards_received' ]>0).sum()
    return post
count= Awarded_posts(df)
print(count)

119


Find Covid

In [24]:
def covidcorona_subreddits(df):
  sub=df['subreddit'].str.startswith(('Covid', 'Corona'))
  desc=df['subr_description'].str.contains('covid', case=False)
  all=df[sub & desc]
  dict1=all.set_index('subreddit')['subr_description'].to_dict()
  return dict1

print(covidcorona_subreddits(df))

{'Coronavirus': 'Place to discuss all things COVID-related', 'CoronavirusCA': 'Tracking the Coronavirus/Covid-19 outbreak in California', 'CoronavirusUS': 'USA/Canada specific information on the coronavirus (SARS-CoV-2) that causes coronavirus disease 2019 (COVID-19)', 'CoronavirusDownunder': 'This subreddit is a place to share news, information, resources, and support that relate to the novel coronavirus SARS-CoV-2 and the disease it causes called COVID-19. The primary focus of this sub is to actively monitor the situation in Australia, but all posts on international news and other virus-related topics are welcome, to the extent they are beneficial in keeping those in Australia informed.'}


Redditors that favorite the most

In [25]:
def count_favreddits_by_user(df):
   temp=df.copy()
   temp=temp.explode('subr_faved_by_as_list')
   grouped=temp.groupby('subr_faved_by_as_list')['subreddit'].nunique()
   result= grouped.reset_index()
   result.columns= ['redditor', 'numb_favs']
   result=result.sort_values('numb_favs', ascending=False).reset_index(drop=True)
   return result
new_table=count_favreddits_by_user(df)
print(new_table)

              redditor  numb_favs
0       magnusthered15          7
1          OmniusQubus          6
2             ry_ta506          6
3            KarmaFury          6
4             SaneFive          6
...                ...        ...
1593  vergil_never_cry          1
1594   very_zesty_zach          1
1595          vi_carel          1
1596         vizard673          1
1597    vlone_Disciple          1

[1598 rows x 2 columns]
